## Enhanced Frost filter

##### The **Enhanced Frost Filter (EFF)** is an adaptive filter developed in order to remove multiplicative 'noise', which in this case is the SAR speckle (note that speckle is not noise!). The filter makes use of the local statistics for providing an estimate of the minimum mean square error (MMSE) inside a homogenous area. The filter is used for applications where edge preservation is desired. EFF is an alteration of the classical Frost filter, and uses the same principle of local statistics (mean and variance), namely the use of the **coefficient of variation** calculated in each window. An aproximation of the 'grey level' for each pixel is done by computing the weighted sum of the center pixel value, the mean value and the variance in the kernel. Pixels are then assigned one of the three classes:
 
 - Homogenous: the new pixel value is the value of the average within the filter window;
 - Heterogenous: the new pixel value is the value of the weighted average;
 - Point target: the new pixel value is the same as the old pixel value. 
 
The original implementation of the Frost filter is described in Frost, V., et al. (1982) A Model for Radar Images and Its Application to Adaptive Digital Filtering of Multiplicative Noise, IEEE Transactions on Pattern Analysis and Machine Intelligence, Vol PAMI-4, No. 2. The Enhanced Frost implementation follows  the algorithm explanation described by Lopez, A., et al., (1990), Adaptive Speckle Filters and Scene Heterogeneity, IEEE Transactions on Geoscience and Remote Sensing, Vol. 28, No.6. 


In [2]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
from skimage import exposure
import os
import glob

In [3]:
def eff_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape

    
    #define the filter prerequisites
    
    window_size= 5 #default value- can be changed with an odd value
    nlooks= 4.4 #default value for Sentinel-1 senzor (according to technical documentation), can be modified
    c= 1 #default value for normalizing constant 
    cu = 1 / math.sqrt(nlooks)
    cmax = math.sqrt(1 + 2 / nlooks)
    
    mean=cv.blur(img_array, (window_size, window_size)) #calculate the mean of the local window
    mean2=cv.blur(img_array*img_array, (window_size, window_size))
    loc_var=mean2-mean*mean #calculate the variance in the local window
    ci=loc_var*loc_var/mean*mean
    
    
    #the decaying constant (alpha) as defined in the original paper. Note that other implementations follow a shorter path of using a general dampening factor which can substitute the calculation of alpha. 
    
    def alpha():
        a=np.zeros(img_array.shape, dtype=np.float32)
        h,w= img_array.shape
        for i in range(0,h):
            for j in range (0,w):
                 a[i,j]=math.sqrt(c*ci[i,j])
        return a

    alpha=alpha()
    
    #calculating the matrix of values for estimating position of different pixels relative to the center pixel in the local window
    def distance_matrix(window_size):
        dist_matrix = np.ndarray((window_size, window_size), dtype=np.float32)
        center_idx = window_size // 2
        for i in range(window_size):
                for j in range(window_size):
                        dist_matrix[i, j] = np.abs(np.sqrt(np.power(i - center_idx, 2) + np.power(j - center_idx, 2)))
        return dist_matrix
    
    distances=distance_matrix(window_size)
    
    # define filter function
    
    def enhanced_frost(window_size, array):
        output=np.zeros(img_array.shape, dtype=np.float32)
        h,w=img_array.shape
        for m in range (window_size):
            for n in range(window_size):
                for i in range(0,h):
                    for j in range(0,w):
                        if ci[i,j]<=cu:
                            output[i,j]= mean[i,j] 
                        elif ci[i,j]>=cmax: 
                            output[i,j]=img_array[i,j]
                        else:
                            output[i,j]=c*alpha[i,j]*(np.exp(-alpha[i,j]*distances[m,n]))
            return output
    
    #filter image
    
    filtered_image=enhanced_frost(window_size, img_array)
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [5]:
sar_image1=gdal.Open('/home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI1.tif')
sar_band1=sar_image1.GetRasterBand(1)
sar_band1.GetMetadata()
img_array1=sar_band1.ReadAsArray()
[cols, rows]=img_array1.shape

In [8]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/8bit/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_EFF_5.tif'
    output_file = output_file[:-2] + ['Filters/Enhanced_Frost'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    eff_filter(input_file, output_file)

Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI1.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Frost/8bit/ROI1_EFF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI1.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI3.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Frost/8bit/ROI3_EFF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI3.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI4.tif
Created file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/Filters/Enhanced_Frost/8bit/ROI4_EFF_5.tif
End filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI4.tif
Begin filtering file /home/cristina/Seeps/Workflow/Filters/ROI/Rize_20_07_2019/8bit/ROI2.tif
Created file 